In [29]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [30]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [31]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [32]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [33]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [34]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [35]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['text'])
    data["text"] = data["text"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["text"] = data['text'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["text"] = data["text"].apply(remove_punctuations)
    data["text"] = data['text'].str.replace(r'\d+', '', regex=True)
    data["text"] = data["text"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["text"] = data["text"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["text"]

In [36]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [37]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 0:
        return 'negative'
    else:
        return 'positive'

In [38]:
txt = "awesome product. i love it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'

In [39]:
txt = "fuck you bitch"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'negative'

In [40]:
txt = "i don't like you"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'negative'

In [41]:
txt = "The project is missing proper documentation, making it difficult for others to understand how to set up and use the system. Clear instructions would greatly improve usability."
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'negative'

In [42]:
txt = "The code is hard to read due to inconsistent formatting and lack of comments. Maintaining a uniform style and adding inline comments will make it easier for others to maintain and extend the code."
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'negative'

In [44]:
txt = "excellent"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'

In [45]:
txt = "you are perfect"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'